## Fixed-Income Pricing Model Delovpment And Validation 

### Pricing Model: DCF | Asset Class: Fixed Income | Finacial Product: US Treasury Bond

#### Import Liabaries

In [3]:
import pandas as pd
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from datetime import date, timedelta

In [7]:
dayCountMapping = {'1 Mo': 30, '2 Mo': 60, '3 Mo': 90, '4 Mo': 120, '6 Mo': 180, '1 Yr': 360, 
                   '2 Yr': 720, '3 Yr': 1080, '5 Yr': 1800, '7 Yr': 2520, '10 Yr': 3600, '20 Yr': 7200, '30 Yr': 10800}

#### Instrument Trade Data

In [69]:
instrumentType = 'Bond'
CUSIP = '912810TX6'

In [71]:
trade_data = pd.read_csv(fr"C:\Users\dines\Desktop\Market Risk Project\30Y US Treasury {instrumentType}_{CUSIP}\Script {CUSIP}.csv",index_col="CUSIP")

In [12]:
trade_data

,Security Type,Security Term,Auction Date,Issue Date,Maturity Date,Price per $100
CUSIP,,,,,,
912810TX6,Bond,29-Year 11-Month,03/13/2024,03/15/2024,02/15/2054,98.642640
912810TX6,Bond,30-Year,02/08/2024,02/15/2024,02/15/2054,98.168819


#### Insturment Features

In [65]:
CUSIP = trade_data.index[0]
faceValue = 100
cuponRate = 0.0425
maturityPeriod = int(trade_data['Security Term'].values[-1].split("-")[0])
frequency = 2
auctionDate = pd.to_datetime(trade_data['Auction Date'].values[-1]).date()
issueDate = pd.to_datetime(trade_data['Issue Date'].values[-1]).date()
maturityDate = pd.to_datetime(trade_data['Maturity Date'].values[-1]).date()
issuePrice = trade_data['Price per $100'].values[-1]

In [16]:
valuationDate = auctionDate

In [18]:
issueDate

datetime.date(2024, 2, 15)

#### Coupon Dates

In [21]:
Y, M, D = issueDate.year, issueDate.month, issueDate.day

cuponDates = []
for i in range(1,(maturityPeriod*frequency) + 1):
    try:
        if i % 2 != 0:
            cuponDates.append(date(Y, M+6, D))
        else:
            cuponDates.append(date(Y+1, M, D))
    except ValueError:
        cuponDates.append(date(Y+1,M,D-1))
print(cuponDates)

[datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025,

In [23]:
cuponDays = [180 * i for i in range(1,(maturityPeriod*frequency) + 1)]
print(cuponDays)

[180, 360, 540, 720, 900, 1080, 1260, 1440, 1620, 1800, 1980, 2160, 2340, 2520, 2700, 2880, 3060, 3240, 3420, 3600, 3780, 3960, 4140, 4320, 4500, 4680, 4860, 5040, 5220, 5400, 5580, 5760, 5940, 6120, 6300, 6480, 6660, 6840, 7020, 7200, 7380, 7560, 7740, 7920, 8100, 8280, 8460, 8640, 8820, 9000, 9180, 9360, 9540, 9720, 9900, 10080, 10260, 10440, 10620, 10800]


## Market Data Preparation

#### Intrest Rate

In [27]:
class InterestRates:

    def __init__(self, Y):
        self.Y = Y

    def USTreasury(self, ratesCurve=False, COBDate=None):
        USTreasuryData = pd.read_csv(f'https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/{self.Y}/all?type=daily_treasury_yield_curve&field_tdr_date_value={self.Y}&page&_format=csv', index_col='Date').iloc[::-1]  
        USTreasuryData.index = pd.to_datetime(USTreasuryData.index)
        
        if ratesCurve == True:
            if COBDate == None:
                return USTreasuryData[USTreasuryData.index == USTreasuryData.index.max()]
            else:
                return USTreasuryData[USTreasuryData.index == COBDate]
        else:
            return USTreasuryData

In [29]:
spotRates = InterestRates(Y=valuationDate.year).USTreasury(ratesCurve=True, COBDate=pd.to_datetime(valuationDate))
spotRates = pd.DataFrame(data=spotRates.values, index=spotRates.index, columns=dayCountMapping.values())

In [30]:
spotRates

,30,60,90,120,180,360,720,1080,1800,2520,3600,7200,10800
Date,,,,,,,,,,,,,
2024-02-08,5.49,5.51,5.44,5.42,5.24,4.83,4.46,4.22,4.12,4.15,4.15,4.47,4.36


####  Interst Rate Determination

In [34]:
tenors , interestRates = spotRates.columns/360 , spotRates.values[0]/100
print(tenors, interestRates)

Index([0.08333333333333333, 0.16666666666666666,                0.25,
        0.3333333333333333,                 0.5,                 1.0,
                       2.0,                 3.0,                 5.0,
                       7.0,                10.0,                20.0,
                      30.0],
      dtype='float64') [0.0549 0.0551 0.0544 0.0542 0.0524 0.0483 0.0446 0.0422 0.0412 0.0415
 0.0415 0.0447 0.0436]


In [36]:
interpfunction = interpolate.CubicSpline(x= tenors, y= interestRates)
tenorsInterp = [t/360 for t in cuponDays]
interpolatedRates = interpfunction(tenorsInterp)
print(interpolatedRates)

[0.0524     0.0483     0.04615596 0.0446     0.04326031 0.0422
 0.0415442  0.04123122 0.04115262 0.0412     0.04128086 0.04136641
 0.04144375 0.0415     0.04152611 0.04152834 0.04151677 0.04150146
 0.04149251 0.0415     0.04153201 0.04158867 0.04166813 0.0417685
 0.04188794 0.04202458 0.04217656 0.04234201 0.04251906 0.04270587
 0.04290055 0.04310126 0.04330612 0.04351327 0.04372085 0.043927
 0.04412986 0.04432755 0.04451822 0.0447     0.04487103 0.04502945
 0.04517339 0.045301   0.0454104  0.04549973 0.04556713 0.04561074
 0.0456287  0.04561913 0.04558019 0.04550999 0.04540669 0.04526842
 0.04509331 0.0448795  0.04462512 0.04432833 0.04398724 0.0436    ]


### Pricing Model

#### Discount Cashflow (DCF) Approach

In [40]:
futurevalueCF = [faceValue*(cuponRate/frequency) for t in range(1,(maturityPeriod*frequency)+1)]
futurevalueCF[-1] += faceValue
print(futurevalueCF)

[2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 2.125, 102.125]


In [42]:
annualizedRate = [(1+i/frequency)**frequency -1 for i in interpolatedRates]
print(annualizedRate)

[0.05308643999999996, 0.04888322249999977, 0.0466885495112519, 0.04509728999999996, 0.04372817411548313, 0.04264520999999988, 0.041975683681502396, 0.04165621848587353, 0.0415760038621511, 0.041624359999999916, 0.041706891072823105, 0.041794200984497376, 0.041873141507501765, 0.04193056250000016, 0.04195721934090524, 0.04195949400742505, 0.041947676003363465, 0.04193205507499731, 0.04192292076926063, 0.04193056250000016, 0.0419632381799262, 0.04202107896924834, 0.042102184873221704, 0.04220465642421556, 0.042326594478083734, 0.042466100027893416, 0.04262127403501448, 0.0427902172775656, 0.04297103021622051, 0.04316181287737342, 0.04336066475366129, 0.043565684721849074, 0.043774970978068595, 0.04398662099042028, 0.04419873146893094, 0.044409398352875495, 0.04461671681544788, 0.04481878128580408, 0.04501368548844997, 0.045199522500000144, 0.04537438482328682, 0.045536364478834246, 0.04568355311368544, 0.04581404212759432, 0.045925922816571596, 0.046017286533792445, 0.046086224867862624,

In [44]:
presentValueCF = [futurevalueCF[i]/(1+interpolatedRates[i]/frequency) ** (i+1) for i in range(maturityPeriod*frequency)]
bondPrice = sum(presentValueCF)
print(bondPrice,presentValueCF)

98.14680544684727 [2.070746443188462, 2.025964334652136, 1.9844160213499322, 1.945563863662673, 1.9093714183128403, 1.8747755337081196, 1.840167644043773, 1.8049338546873814, 1.7690886153540508, 1.733018881424793, 1.6972994883756116, 1.6621389259320067, 1.6276553808729761, 1.5939969793133788, 1.5612942987030816, 1.5295097332644576, 1.4985412306432158, 1.4682646421483978, 1.4385362512636777, 1.4091952235817276, 1.3800943198337459, 1.3511929991330216, 1.3224839508911195, 1.2939658775992628, 1.2656429381740035, 1.2375242011250982, 1.209623108196498, 1.1819569497702729, 1.1545463538343461, 1.127414790703562, 1.1005880959554109, 1.074094014204098, 1.047961766398356, 1.022221643298731, 0.9969046276811806, 0.9720420476365802, 0.947665263104556, 0.9238053875067775, 0.9004930460455269, 0.8777581719194386, 0.8556298413960826, 0.8341361483817453, 0.8133041188568955, 0.7931596653121649, 0.7737275811369252, 0.7550315747903912, 0.7370943435344488, 0.7199376865376111, 0.703582657280112, 0.68804975541

#### Model Error

In [47]:
print('Model Price:',bondPrice)
print('Market Price:',issuePrice)

priceDiff = bondPrice-issuePrice
print('Price Difference:', round(priceDiff,4))

Model Price: 98.14680544684727
Market Price: 98.168819
Price Difference: -0.022


In [49]:
pricingResult =pd.Series({"Model Price":bondPrice,"Issue Price":issuePrice,"Price Difference":priceDiff})

In [53]:
pricingResult.to_csv(f"C:\\Users\\dines\\Desktop\\Market Risk Project\\{maturityPeriod}Y US Treasury {instrumentType}_{CUSIP}\\Pricing RESULTS for {CUSIP}.csv")